In [5]:
from time import gmtime, strftime
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

In [4]:
%pip install --upgrade boto3

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 39.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.46
    Uninstalling boto3-1.26.46:
      Successfully uninstalled boto3-1.26.46
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.46 requires botocore==1.29.46, but you have botocore 1.29.58 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but you have botocore 1.29.58 which is incompatible.
Note: you may need to restart the kernel to use

### Script to train and deploy model to sagemaker
Requires training data to be generated and uploaded to s3
See model training notebook

Deploys to a serverless endpoint that scales on demand, tests show cold start is ~6s, after that subsecond

In [6]:
bucket = 'cb-analytics-us-east-2-prd'
prefix = 'sagemaker/training_data'

In [7]:
region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

AWS Region: us-east-2
RoleArn: arn:aws:iam::257056996471:role/cb-sagemaker


In [13]:
# Train my estimator
# from sagemaker import SKLearn

itc_mse_no_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'mse',
                                'include_current_hours': False
                            },
                            framework_version='1.0-1')

In [14]:
itc_mse_no_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-01-27-20-26-35-150


2023-01-27 20:26:35 Starting - Starting the training job...
2023-01-27 20:26:50 Starting - Preparing the instances for training......
2023-01-27 20:28:03 Downloading - Downloading input data
2023-01-27 20:28:03 Training - Downloading the training image...
2023-01-27 20:28:28 Training - Training image download completed. Training in progress...2023-01-27 20:28:45,901 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-27 20:28:45,904 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:28:45,912 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-27 20:28:46,091 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:28:46,102 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:28:46,112 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:28

### Serverless deploy

In [15]:
model_name = "ds-rec-hrs-itc-no-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

Model name: ds-rec-hrs-itc-no-cur-hrs-serverless-20230127-202916


In [16]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [17]:
itc_mse_no_cur_hrs_predictor = itc_mse_no_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

INFO:sagemaker:Creating model with name: ds-rec-hrs-itc-no-cur-hrs-serverless-20230127-202916
INFO:sagemaker:Creating endpoint-config with name ep-ds-rec-hrs-itc-no-cur-hrs-serverless-20230127-202916
INFO:sagemaker:Creating endpoint with name ep-ds-rec-hrs-itc-no-cur-hrs-serverless-20230127-202916


-----!

In [18]:
itc_mse_no_cur_hrs_predictor.endpoint_name

'ep-ds-rec-hrs-itc-no-cur-hrs-serverless-20230127-202916'

In [19]:
# Train my estimator
# from sagemaker import SKLearn

itc_mse_w_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'mse',
                                'include_current_hours': True
                            },
                            framework_version='1.0-1')

In [20]:
itc_mse_w_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-01-27-20-36-51-528


2023-01-27 20:36:52 Starting - Starting the training job...
2023-01-27 20:37:06 Starting - Preparing the instances for training......
2023-01-27 20:38:18 Downloading - Downloading input data
2023-01-27 20:38:18 Training - Downloading the training image...
2023-01-27 20:38:43 Training - Training image download completed. Training in progress...2023-01-27 20:39:02,067 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-27 20:39:02,071 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:39:02,079 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-27 20:39:02,298 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:39:02,311 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:39:02,327 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 20:39

In [ ]:
### Serverless deploy

In [21]:
model_name = "ds-rec-hrs-itc-w-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

Model name: ds-rec-hrs-itc-w-cur-hrs-serverless-20230127-203933


In [22]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [23]:
itc_mse_w_cur_hrs_predictor = itc_mse_w_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

INFO:sagemaker:Creating model with name: ds-rec-hrs-itc-w-cur-hrs-serverless-20230127-203933
INFO:sagemaker:Creating endpoint-config with name ep-ds-rec-hrs-itc-w-cur-hrs-serverless-20230127-203933
INFO:sagemaker:Creating endpoint with name ep-ds-rec-hrs-itc-w-cur-hrs-serverless-20230127-203933


-----!

In [24]:
itc_mse_w_cur_hrs_predictor.endpoint_name

'ep-ds-rec-hrs-itc-w-cur-hrs-serverless-20230127-203933'

### quantile no cur hrs low

In [25]:
# Train my estimator
# from sagemaker import SKLearn

itc_qtl_no_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'quantile',
                                'quantile': 0.1,
                                'include_current_hours': False
                            },
                            framework_version='1.0-1')

In [26]:
itc_qtl_no_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-01-27-21-28-45-881


2023-01-27 21:28:46 Starting - Starting the training job...
2023-01-27 21:29:02 Starting - Preparing the instances for training......
2023-01-27 21:30:10 Downloading - Downloading input data
2023-01-27 21:30:10 Training - Downloading the training image...
2023-01-27 21:30:35 Training - Training image download completed. Training in progress...2023-01-27 21:30:51,428 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-27 21:30:51,431 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:30:51,438 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-27 21:30:51,604 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:30:51,615 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:30:51,627 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:30

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2023-01-27-21-28-45-881: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/entry_point.py", line 99, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", line 286, in run
    process = check_error(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", 

In [ ]:
### Serverless deploy

In [ ]:
model_name = "ds-rec-hrs-itc-quant-low-no-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

In [ ]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [ ]:
itc_qtl_no_cur_hrs_predictor = itc_qtl_no_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

In [ ]:
itc_qtl_no_cur_hrs_predictor.endpoint_name

### quantile no cur hrs high

In [ ]:
# Train my estimator
# from sagemaker import SKLearn

itc_qth_no_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'quantile',
                                'quantile': 0.9,
                                'include_current_hours': False
                            },
                            framework_version='1.0-1')

In [ ]:
itc_qtl_no_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

In [ ]:
### Serverless deploy

In [ ]:
model_name = "ds-rec-hrs-itc-quant-high-no-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

In [ ]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [ ]:
itc_qth_no_cur_hrs_predictor = itc_qth_no_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

In [ ]:
itc_qth_no_cur_hrs_predictor.endpoint_name

### quantile w cur hrs low

In [27]:
# Train my estimator
# from sagemaker import SKLearn

itc_qtl_w_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'quantile',
                                'quantile': 0.1,
                                'include_current_hours': True
                            },
                            framework_version='1.0-1')

In [28]:
itc_qtl_w_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2023-01-27-21-31-52-569


2023-01-27 21:31:52 Starting - Starting the training job...
2023-01-27 21:32:07 Starting - Preparing the instances for training......
2023-01-27 21:33:18 Downloading - Downloading input data
2023-01-27 21:33:18 Training - Downloading the training image...
2023-01-27 21:33:43 Training - Training image download completed. Training in progress...2023-01-27 21:34:01,616 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-27 21:34:01,619 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:34:01,627 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-27 21:34:01,840 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:34:01,851 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:34:01,862 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-27 21:34

UnexpectedStatusException: Error for Training job sagemaker-scikit-learn-2023-01-27-21-31-52-569: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_sklearn_container/training.py", line 31, in train
    entry_point.run(uri=training_environment.module_dir,
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/entry_point.py", line 99, in run
    return runner.get(runner_type, user_entry_point, args, env_vars, extra_opts).run(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", line 286, in run
    process = check_error(
  File "/miniconda3/lib/python3.8/site-packages/sagemaker_training/process.py", 

In [ ]:
### Serverless deploy

In [ ]:
model_name = "ds-rec-hrs-itc-quant-low-w-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

In [ ]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [ ]:
itc_qtl_w_cur_hrs_predictor = itc_qtl_w_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

In [ ]:
itc_qtl_w_cur_hrs_predictor.endpoint_name

### quantile w cur hrs high

In [ ]:
# Train my estimator
# from sagemaker import SKLearn

itc_qth_w_cur_hrs_est = SKLearn(entry_point='20230127_itc_train_deploy.py',
                            instance_type='ml.m5.xlarge',
                            role=role,
                            hyperparameters={
                                'error': 'quantile',
                                'quantile': 0.9,
                                'include_current_hours': True
                            },
                            framework_version='1.0-1')

In [ ]:
itc_qtl_w_cur_hrs_est.fit({'train': f's3://{bucket}/{prefix}/20230127201843_itc_traning_data.csv'})

In [ ]:
### Serverless deploy

In [ ]:
model_name = "ds-rec-hrs-itc-quant-high-w-cur-hrs-serverless-" + strftime("%Y%m%d-%H%M%S", gmtime())
print("Model name: " + model_name)

In [ ]:
serverless_conf = sagemaker.serverless.serverless_inference_config.ServerlessInferenceConfig(
    memory_size_in_mb=6144,
    max_concurrency=100
)

In [ ]:
itc_qth_w_cur_hrs_predictor = itc_qth_w_cur_hrs_est.deploy(serverless_inference_config=serverless_conf,
                                     endpoint_name='ep-' + model_name,
                                     model_name=model_name
                                    )

In [ ]:
itc_qth_w_cur_hrs_predictor.endpoint_name